In [7]:
from sklearn import datasets
import pandas as pd
import numpy as np

# Загружаем датасет
iris = datasets.load_iris()

# Конвертируем в dataframe
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                  columns= iris['feature_names'] + ['target'])

# FФильтруем по 2м признакам Versicolor и Virginica
df = df[df.target != 0]


Реализуем LogisticRegression

In [2]:
class LogisticRegression:
    def __init__(self, learning_rate=0.01, num_iterations=10000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.weights = np.zeros(num_features)
        self.bias = 0

        for _ in range(self.num_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self._sigmoid(linear_model)

            dw = (1 / num_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / num_samples) * np.sum(y_predicted - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        return y_predicted_cls


Проводим обучение модели метедом LogisticRegression

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = df.drop(columns='target').values
y = df['target'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print('Accuracy: ', accuracy)


Accuracy:  0.6


Реализуем LogisticRegressionRMSProp

In [9]:
class LogisticRegressionRMSProp:
    def __init__(self, learning_rate=0.01, num_iterations=10000, eps=1e-8, decay_rate=0.9):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.eps = eps
        self.decay_rate = decay_rate
        self.weights = None
        self.bias = None

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.weights = np.zeros(num_features)
        self.bias = 0

        # Initialize parameters for RMSProp
        weight_cache = np.zeros(num_features)
        bias_cache = 0

        for _ in range(self.num_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self._sigmoid(linear_model)

            dw = (1 / num_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / num_samples) * np.sum(y_predicted - y)

            # Update cache with new weights
            weight_cache = self.decay_rate * weight_cache + (1 - self.decay_rate) * dw**2
            bias_cache = self.decay_rate * bias_cache + (1 - self.decay_rate) * db**2

            # Update weights and bias using RMSProp update rule
            self.weights -= (self.learning_rate / (np.sqrt(weight_cache) + self.eps)) * dw
            self.bias -= (self.learning_rate / (np.sqrt(bias_cache) + self.eps)) * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        return y_predicted_cls


Проводим обучение модели метедом LogisticRegressionRMSProp

In [10]:
X = df.drop(columns='target').values
y = df['target'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegressionRMSProp()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print('Accuracy: ', accuracy)

Accuracy:  0.6


Реализуем LogisticRegressionNadam

In [11]:
class LogisticRegressionNadam:
    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, num_iterations=1000):
        self.learning_rate = learning_rate
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.epsilon = epsilon
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        num_samples, num_features = X.shape

        self.weights = np.zeros(num_features)
        self.bias = 0

        m_w, v_w = np.zeros(num_features), np.zeros(num_features)
        m_b, v_b = 0, 0
        for i in range(self.num_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self._sigmoid(linear_model)

            dw = (1 / num_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / num_samples) * np.sum(y_predicted - y)

            m_w = self.beta_1 * m_w + (1. - self.beta_1) * dw
            v_w = self.beta_2 * v_w + (1. - self.beta_2) * np.square(dw)
            m_hat_w = m_w / (1. - self.beta_1 ** (i + 1))
            v_hat_w = v_w / (1. - self.beta_2 ** (i + 1))

            m_b = self.beta_1 * m_b + (1. - self.beta_1) * db
            v_b = self.beta_2 * v_b + (1. - self.beta_2) * np.square(db)
            m_hat_b = m_b / (1. - self.beta_1 ** (i + 1))
            v_hat_b = v_b / (1. - self.beta_2 ** (i + 1))

            self.weights -= self.learning_rate * (self.beta_1 * m_hat_w + (1. - self.beta_1) * dw / (1. - self.beta_1 ** (i + 1))) / (np.sqrt(v_hat_w) + self.epsilon)
            self.bias -= self.learning_rate * (self.beta_1 * m_hat_b + (1. - self.beta_1) * db / (1. - self.beta_1 ** (i + 1))) / (np.sqrt(v_hat_b) + self.epsilon)

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        return np.array(y_predicted_cls)


Проводим обучение модели метедом LogisticRegressionNadam

In [12]:
X = df.drop(columns='target').values
y = df['target'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegressionNadam()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print('Accuracy: ', accuracy)

Accuracy:  0.6


Сравниваем значение метрик для реализованных методов оптимизации

In [13]:
from sklearn.metrics import accuracy_score
from time import time

# Инициализация моделей
models = [
    LogisticRegression(learning_rate=0.1, num_iterations=10000),
    LogisticRegressionRMSProp(learning_rate=0.1, num_iterations=10000),
    LogisticRegressionNadam(learning_rate=0.1, num_iterations=10000)
]

# Инициализация переменных для сохранения результатов
results = []
model_names = ['Gradient Descent', 'RMSProp', 'Nadam']

# Обучение моделей и измерение точности и времени
for i, model in enumerate(models):
    start_time = time()
    model.fit(X_train, y_train)
    end_time = time()
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    time_elapsed = end_time - start_time
    results.append([model_names[i], accuracy, time_elapsed])

# Создание DataFrame с результатами
results_df = pd.DataFrame(results, columns=['Method', 'Accuracy', 'Time'])

print(results_df)


             Method  Accuracy      Time
0  Gradient Descent       0.6  0.135889
1           RMSProp       0.6  0.174940
2             Nadam       0.6  0.240683


Вывод: модели дали одинаковую точность, но метод градиентного спуска сработал быстрее всего.